## How to reindex a collection
This notebook will walk through the process of ingesting a dataset and loading a collection in Milvus. After the collection is ingested, we will show how a user, can reindex a collection using the `reindex_collection` function. With this the user is able to grab all data from the collection itself, in the case that the user does not have access to the original corpus. When reindexing a collection, all collection related metadata will be conserved with each element. This function, pulls all the data the identified collection and 

First step is to annotate all the necessary variables to ensure our client connects to our pipeline.

In [ ]:
model_name="nvidia/llama-3.2-nv-embedqa-1b-v2"
hostname="localhost"
collection_name = "nv_ingest_collection"
sparse = True

Next step, instantiate your ingestor object with all the stages you want in your pipeline. Ensure that you have a vdb_upload stage, as this is what will load your transformed elements(data) in to the vector database. 

In [ ]:
from nv_ingest_client.client import Ingestor

ingestor = ( 
    Ingestor(message_client_hostname=hostname)
    .files(["data/woods_frost.pdf", "data/multimodal_test.pdf"])
    .extract(
        extract_text=True,
        extract_tables=True,
        extract_charts=True,
        extract_images=True,
        text_depth="page"
    ).embed()
    .vdb_upload(
        collection_name=collection_name, 
        milvus_uri=f"http://{hostname}:19530", 
        sparse=sparse, 
        minio_endpoint=f"{hostname}:9000", 
        dense_dim=2048
    )
)
results = ingestor.ingest()

Once you have completed the normal ingestion, the collection will have been loaded into your Vector Database. If you need to reindex that data for whatever reason, you can simply run the `reindex_collection` function and supply the necessary parameters. There is a full list of parameters in the docstring of the function, with many defaults already set for you. This function is desigend to be used when the results from your ingestor pipeline are no longer available. You might have ingested this information at a previous date/time and the ingestor results are no longer in memory. This function allows you to query the data from the vector database to recreate those results and send them into a new collection or the same collection, effectively replacing the previous information stored in that collection. 

In this example we will reindex under the same collection name, replacing the data in the collection. You can always supply a `new_collection_name` as one of the arguments to the function allowing you to save the reindex in another collection. The function supplies a `write_dir` parameter which allows you to pull the data from the collection and write it into files in batches, relieving memory pressure. Currently the batch_size is automatically set to the default query batch_size for the vector database. The `write_dir` option is meant to be used when the data is larger than the available resources, with this option reindexing is slower than when holding the data in host memory.  

In [ ]:
import os
from nv_ingest_client.util.milvus import reindex_collection

reindex_collection(
    nvidia_api_key=os.environ["NVIDIA_BUILD_API_KEY"],
    collection_name=collection_name,
    sparse=sparse
    )